# Microbial Survival and Growth Simulations

This notebook contains two main parts:

1. **Dried blood decay model** — simulates viability decay of microbes in dried blood under cold and dry vs moderately humid conditions.
2. **Cloth with blood stain model** — simulates microbial growth on a cloth that retains some moisture, at 2°C under different ambient humidities.

These are *illustrative models* with tunable parameters; they are not species-specific, but they demonstrate how temperature and humidity affect microbial dynamics.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random, math

plt.rcParams["figure.figsize"] = (8,4)

# --- DRIED BLOOD DECAY MODEL ---

def temp_preserve_factor(T, T_ref=25.0, Q10=2.0):
    temp_factor = Q10 ** ((T - T_ref) / 10.0)
    return 1.0 - temp_factor  # low T => strong preservation

def death_rate(T, RH, a=0.2, b=0.8):
    tp = temp_preserve_factor(T)
    return a * (1 - tp) + b * (1 - RH/100.0)

def deterministic_decay(N0, d, t_days):
    t = np.linspace(0, t_days, 300)
    N = N0 * np.exp(-d * t)
    return t, N

# Conditions
conds = [
    {"T":2.0, "RH":0.0, "label":"2°C, 0% RH"},
    {"T":2.0, "RH":45.0, "label":"2°C, 45% RH"},
]

N0 = int(1e6)
t_days = 30.0
results = []

for c in conds:
    d = death_rate(c["T"], c["RH"], a=0.2, b=0.8)
    t_det, N_det = deterministic_decay(N0, d, t_days)
    p_survive = math.exp(-d * t_days)
    prob_all_extinct = (1 - p_survive) ** N0
    expected = N0 * p_survive
    results.append((c["label"], d, t_det, N_det, p_survive, expected, prob_all_extinct))

# Plot
plt.figure()
for label, d, t, N, p, exp, prob0 in results:
    plt.plot(t, N, label=f"{label} (d={d:.3f} day⁻¹)")
plt.yscale('log')
plt.xlabel("Time (days)"); plt.ylabel("Viable cells (log)")
plt.title("Viable count in dried blood (decay model)")
plt.legend(); plt.grid(True, which='both', ls='--', lw=0.5)
plt.show()

for label, d, t, N, p, exp, prob0 in results:
    print(f"Condition: {label}\n  Death rate={d:.3f}/day, Survival p={p:.3e}, Expected survivors={exp:.2f}, Extinction prob={prob0:.3e}\n")


In [ ]:
# --- CLOTH WITH BLOOD STAIN MODEL ---

def q10_temp_factor(T, T_ref=25.0, Q10=2.0):
    return Q10 ** ((T - T_ref) / 10.0)

def local_rh(ambient_rh):
    return min(100.0, ambient_rh + 20.0)  # cloth retains +20% RH

def simulate_cloth_logistic(N0=100.0, r_base=0.8, K=5e5, T=2.0, ambient_RH=0.0, t_max=30.0, dt=0.05):
    rh_local = local_rh(ambient_RH)
    temp_fac = q10_temp_factor(T)
    if rh_local < 20.0:
        moisture_fac = 0.02
    else:
        moisture_fac = rh_local / 100.0
    mult = temp_fac * moisture_fac
    r_env = r_base * mult
    t = np.arange(0, t_max+dt, dt)
    N = np.zeros_like(t)
    N[0] = N0
    for i in range(1, len(t)):
        dN = r_env * N[i-1] * (1 - N[i-1]/K)
        if rh_local < 30.0:
            dN -= 0.01 * N[i-1]
        N[i] = max(0.0, N[i-1] + dN * dt)
    return t, N, rh_local, r_env

conds = [
    {"ambient_RH":0.0, "label":"2°C, 0% RH (cloth local RH)"},
    {"ambient_RH":45.0, "label":"2°C, 45% RH (cloth local RH)"},
]

results = []
for c in conds:
    t, N, rh_local, r_env = simulate_cloth_logistic(ambient_RH=c["ambient_RH"])
    results.append((c["label"], t, N, rh_local, r_env))

plt.figure()
for label, t, N, rh_local, r_env in results:
    plt.plot(t, N, label=f"{label} (local RH={rh_local:.0f}%, r_env={r_env:.3e})")
plt.yscale('log')
plt.xlabel("Time (days)"); plt.ylabel("Population (log)")
plt.title("Microbial population on cloth with blood stain")
plt.legend(); plt.grid(True, which='both', ls='--', lw=0.5)
plt.show()

for label, t, N, rh_local, r_env in results:
    print(f"Condition: {label}\n  Local RH={rh_local:.1f}%, r_env={r_env:.4e}\n")
